In [1]:
import csv
import sys
import os

def truncate_csv(input_file, output_file, max_length=255):
    # Track statistics
    truncation_count = 0
    processed_rows = 0
    
    print(f"Reading from: {input_file}")
    print(f"Writing to: {output_file}")
    
    # Check if input file exists
    if not os.path.exists(input_file):
        print(f"Error: Input file '{input_file}' does not exist!")
        return
    
    try:
        with open(input_file, 'r', newline='', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            
            # Get header row (already in lowercase)
            header = next(reader)
            
            with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
                writer = csv.writer(outfile)
                
                # Write the header (no changes needed)
                writer.writerow(header)
                
                # Process each row
                for row in reader:
                    processed_rows += 1
                    new_row = []
                    
                    for value in row:
                        if len(value) > max_length:
                            truncation_count += 1
                            value = value[:max_length-3] + '...'
                        new_row.append(value)
                    
                    writer.writerow(new_row)
        
        print(f"CSV processing complete:")
        print(f"- Processed {processed_rows} rows")
        print(f"- Truncated {truncation_count} values that exceeded {max_length} characters")
        print(f"- Output saved to {output_file}")
    
    except Exception as e:
        print(f"Error processing CSV: {e}")

# Use these filenames - paste.txt for input and cdr_trials_cleaned.csv for output
input_file = "mdr_trials_cleaned.csv"  
output_file = "cdr_trials_cleaned.csv"  

truncate_csv(input_file, output_file)

Reading from: mdr_trials_cleaned.csv
Writing to: cdr_trials_cleaned.csv
CSV processing complete:
- Processed 56 rows
- Truncated 43 values that exceeded 255 characters
- Output saved to cdr_trials_cleaned.csv


In [19]:
import csv
import sys
import os
import re
import string
import traceback

def clean_for_supabase(input_file, output_file, max_length=255):
    # Track statistics
    truncation_count = 0
    cleaning_count = 0
    punctuation_count = 0
    array_fields_count = 0
    processed_rows = 0
    
    print(f"Reading from: {input_file}")
    print(f"Writing to: {output_file}")
    
    # Check if input file exists
    if not os.path.exists(input_file):
        print(f"Error: Input file '{input_file}' does not exist!")
        return
    
    # List of columns that should be formatted as arrays
    array_columns = [
        'organization_type', 
        'other_leading_organization', 
        'all_cdr_methods', 
        'collaborators', 
        'mrv_provider', 
        'monitoring_platforms', 
        'measurements'
    ]
    
    # Create a translation table to remove punctuation except commas for array values
    translator = str.maketrans('', '', string.punctuation.replace(',', ''))
    
    def clean_regular_string(value):
        """Clean a regular (non-array) string field"""
        if not isinstance(value, str):
            return value, False, False, False
        
        if not value.strip():
            return "", False, False, False
            
        cleaned = 0
        has_punctuation = False
        
        # Remove all punctuation
        if any(p in value for p in string.punctuation):
            original_value = value
            value = value.translate(str.maketrans('', '', string.punctuation))
            has_punctuation = True
        
        # Replace control characters and null bytes
        value = re.sub(r'[\x00-\x1F\x7F]', '', value)
        
        # Normalize whitespace
        value = re.sub(r'\s+', ' ', value).strip()
        
        # Truncate if needed
        if len(value) > max_length:
            value = value[:max_length-3] + '...'
            return value, True, cleaned > 0, has_punctuation
            
        return value, False, cleaned > 0, has_punctuation
    
    def clean_array_string(value):
        """Clean and format an array field with curly brackets and quotes"""
        if not isinstance(value, str):
            return "[]"
        
        if not value.strip():
            return "[]"
            
        # Split by commas, clean each item
        items = [item.strip() for item in value.split(',')]
        cleaned_items = []
        
        for item in items:
            if item:  # Skip empty items
                # Remove any existing quotes
                item = item.replace('"', '').replace("'", '')
                
                # Remove punctuation except commas
                cleaned_item = item.translate(translator)
                
                # Normalize whitespace
                cleaned_item = re.sub(r'\s+', ' ', cleaned_item).strip()
                
                # Truncate if needed
                if len(cleaned_item) > max_length:
                    cleaned_item = cleaned_item[:max_length-3] + '...'
                    nonlocal truncation_count
                    truncation_count += 1
                
                # Only add non-empty items
                if cleaned_item:
                    cleaned_items.append(f'"{cleaned_item}"')
        
        # Format as Postgres array with curly braces
        if not cleaned_items:
            return "[]"
        return "{" + ",".join(cleaned_items) + "}"
    
    try:
        with open(input_file, 'r', newline='', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            
            # Get header row
            header = next(reader)
            
            # Map column names to their indices
            column_indices = {name.lower(): i for i, name in enumerate(header)}
            
            # Find indices of array columns
            array_indices = []
            for array_col in array_columns:
                if array_col in column_indices:
                    array_indices.append(column_indices[array_col])
                else:
                    print(f"Warning: Array column '{array_col}' not found in header")
            
            with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
                writer = csv.writer(outfile)
                
                # Write the header
                writer.writerow(header)
                
                # Process each row
                for row in reader:
                    processed_rows += 1
                    new_row = list(row)  # Make a copy we can modify
                    
                    # Process each cell in the row
                    for i in range(len(row)):
                        value = row[i]
                        if i in array_indices:
                            # This is an array field
                            new_row[i] = clean_array_string(value)
                            array_fields_count += 1
                        else:
                            # Regular field
                            clean_value, was_truncated, was_cleaned, had_punctuation = clean_regular_string(value)
                            if was_truncated:
                                truncation_count += 1
                            if was_cleaned:
                                cleaning_count += 1
                            if had_punctuation:
                                punctuation_count += 1
                            new_row[i] = clean_value
                    
                    writer.writerow(new_row)
        
        print(f"CSV processing complete:")
        print(f"- Processed {processed_rows} rows")
        print(f"- Formatted {array_fields_count} array fields with curly brackets")
        print(f"- Removed punctuation from {punctuation_count} values")
        print(f"- Truncated {truncation_count} values that exceeded {max_length} characters")
        print(f"- Cleaned {cleaning_count} values with problematic characters")
        print(f"- Output saved to {output_file}")
    
    except Exception as e:
        print(f"Error processing CSV: {e}")
        print(f"Exception details: {type(e).__name__}: {str(e)}")
        print(f"Line number: {sys.exc_info()[2].tb_lineno}")
        try:
            print(f"Full traceback: {traceback.format_exc()}")
        except:
            pass

# Use these filenames - adjust as needed
input_file = "mdr_trials_cleaned.csv"  
output_file = "cdr_trials_cleaned.csv"  

clean_for_supabase(input_file, output_file)

Reading from: mdr_trials_cleaned.csv
Writing to: cdr_trials_cleaned.csv
CSV processing complete:
- Processed 56 rows
- Formatted 392 array fields with curly brackets
- Removed punctuation from 91 values
- Truncated 22 values that exceeded 255 characters
- Cleaned 0 values with problematic characters
- Output saved to cdr_trials_cleaned.csv


In [28]:
import re
import sys

import csv
import re

import csv
import re

def fix_postgres_arrays(input_file, output_file):
    # Track statistics
    fixed_fields = 0
    
    print(f"Reading from: {input_file}")
    print(f"Writing to: {output_file}")
    
    try:
        with open(input_file, 'r', newline='', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            header = next(reader)
            
            with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
                writer = csv.writer(outfile, quoting=csv.QUOTE_MINIMAL)
                writer.writerow(header)
                
                for row in reader:
                    new_row = []
                    for cell in row:
                        # Check if the cell looks like an array (starts with { and ends with })
                        if cell.startswith('{"') and cell.endswith('"}'):
                            # Split by "," to get individual array elements
                            elements = re.findall(r'"([^"]*)"', cell)
                            
                            # Reconstruct properly formatted PostgreSQL array
                            formatted_array = '{' + ','.join(f'"{element}"' for element in elements) + '}'
                            
                            new_row.append(formatted_array)
                            fixed_fields += 1
                        else:
                            new_row.append(cell)
                    
                    writer.writerow(new_row)
        
        print(f"Array format fixing complete:")
        print(f"- Ensured {fixed_fields} array fields are properly formatted")
        print(f"- Output saved to {output_file}")
    
    except Exception as e:
        print(f"Error processing CSV: {e}")

input_file = "cdr_trials_cleaned.csv"  
output_file = "fixed_cdr_trials.csv"  

fix_postgres_arrays(input_file, output_file)

Reading from: cdr_trials_cleaned.csv
Writing to: fixed_cdr_trials.csv
Array format fixing complete:
- Ensured 305 array fields are properly formatted
- Output saved to fixed_cdr_trials.csv


In [29]:
import csv
import os

def truncate_csv_fields(input_file, output_file, max_length=255):
    # Track statistics
    truncated_fields = 0
    processed_rows = 0
    
    print(f"Reading from: {input_file}")
    print(f"Writing to: {output_file}")
    
    try:
        with open(input_file, 'r', newline='', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            header = next(reader)
            
            with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
                writer = csv.writer(outfile)
                writer.writerow(header)
                
                for row in reader:
                    processed_rows += 1
                    new_row = []
                    
                    for cell in row:
                        # Check if the cell exceeds max length
                        if len(cell) > max_length:
                            # Truncate and add ellipsis
                            truncated_cell = cell[:max_length-3] + "..."
                            new_row.append(truncated_cell)
                            truncated_fields += 1
                        else:
                            new_row.append(cell)
                    
                    writer.writerow(new_row)
        
        print(f"Truncation complete:")
        print(f"- Processed {processed_rows} rows")
        print(f"- Truncated {truncated_fields} fields to {max_length} characters or less")
        print(f"- Output saved to {output_file}")
    
    except Exception as e:
        print(f"Error processing CSV: {e}")

# Use these filenames
input_file = "mdr_trials_cleaned.csv"
output_file = "mdr_trials_truncated.csv"

truncate_csv_fields(input_file, output_file)

Reading from: mdr_trials_cleaned.csv
Writing to: mdr_trials_truncated.csv
Truncation complete:
- Processed 56 rows
- Truncated 43 fields to 255 characters or less
- Output saved to mdr_trials_truncated.csv
